<a href="https://colab.research.google.com/github/namiyousef/argument-mining/blob/develop/examples/OpenModelExperimentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Open Model Experimentation

This notebook allows you to access testing data used for the project, as well as trained models. The original training data is not hosted here.

The document will first take you through steps to set-up the notebook so that you can access the data.

## Colab Set up

Firstly, please access the following [link](https://drive.google.com/drive/folders/1LzSEc25qZHSB5snig1Ro_pgZOnmZ9p2W?usp=sharing) to get access to the drive. This contains a folder named test/ that includes adversarial examples and a folder named tmpdir/ that includes the trained models. You can look into these in greater detail, however the names are not user friendly. These models are identical to those pushed on HuggingFace under https://huggingface.co/ucabqfe.

Once you have done that, please go on your drive under "Shared with me", right click the Desktop/ folder and select "Add shortcut to drive". This will allow you to access the data from within Colab. You can find more information on this here: https://github.com/namiyousef/argument-mining/issues/38. You can ignore the aspects on Authentication as the repository has now made public and published as a package.

In [16]:
!pip install colab-dev-tools

import os
from colabtools.utils import mount_drive, get_gpu_utilization

# get the path to your drive, e.g. drive/MyDrive
DRIVE_PATH = mount_drive()


# set paths
PATH_TO_MODELS = os.path.join(DRIVE_PATH, 'Desktop/tmpdir/job')
PATH_TO_ADVERSARIAL = os.path.join(DRIVE_PATH, 'Desktop/test')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Google Drive mount successful.


In [30]:
!pip install argminer

# -- public imports

import gc
from transformers import AutoTokenizer, AutoModelForTokenClassification
import pandas as pd
from torch.utils.data import DataLoader
import torch
from pandas.testing import assert_frame_equal
import time

# -- private imports
from colabtools.utils import move_to_device
from colabtools.config import DEVICE

import argminer
from argminer.data import ArgumentMiningDataset, TUDarmstadtProcessor, PersuadeProcessor
from argminer.evaluation import inference, _get_scores_agg
from argminer.utils import encode_model_name
from argminer.config import LABELS_MAP_DICT, MODEL_MAP_DICT

# Testing on Adversarial Examples

In [33]:
BATCH_SIZE = 64
MODEL_NAME = 'ucabqfe/roberta_AAE_io' #   select huggingface model name (this is one of our models, see ucabqfe/ models on HuggingFace)
# alternatively, you should be able to specify to a path on the trained models. You can decode the names of the models using argminer.utils.decode_model_name()

model_metadata = MODEL_MAP_DICT[MODEL_NAME]
dataset = model_metadata['dataset']
max_length = model_metadata['max_length']
tokenizer_name = model_metadata['hugging_face_model_name']
strategy = MODEL_NAME.split('_')[-1]
print(f'Running model: {MODEL_NAME}:')
print('=======================================================================')
PATH_TO_ATTACK = os.path.join(PATH_TO_ADVERSARIAL, strategy)
for adversarial_attack in os.listdir(PATH_TO_ATTACK):
  model = AutoModelForTokenClassification.from_pretrained(MODEL_NAME)
  tokenizer = AutoTokenizer.from_pretrained(tokenizer_name, add_prefix_space=True)

  Processor = getattr(argminer.data, f'{dataset}Processor')
  processor = Processor(os.path.join(PATH_TO_ATTACK, adversarial_attack)).from_json(status='postprocessed')
  df_text = processor.dataframe[['text', 'labels']]
  df_label_map = LABELS_MAP_DICT[dataset][strategy]
  dataset = ArgumentMiningDataset(df_label_map, df_text, tokenizer, max_length, f'standard_{strategy}', is_train=False)
  dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=False)

  df_metrics, df_scores = inference(model, dataloader)
  metrics, df_scores_agg = _get_scores_agg(df_scores)
  print(f'Completed inference on adversarial attack: {adversarial_attack}. Macro Metrics: {metrics}\n')
  print(df_scores_agg.to_string(), '\n\n')


Running model: ucabqfe/roberta_AAE_io:
Prediction time: 0.0292
Agg to word time: 10.8
Get predstring time: 0.778
Evaluate time: 3.87
Batch 1 complete.
Prediction time: 0.0282
Agg to word time: 3.02
Get predstring time: 0.172
Evaluate time: 0.81
Batch 2 complete.
Completed inference on adversarial attack: spellingError. Macro Metrics: {'macro_f1': 0.5816210181066526, 'macro_recall': 0.6365550482985234, 'macro_precision': 0.5411039776205414}
        tp   fn   fp        f1    recall  precision
class                                              
0      667  620  681  0.506262  0.518260   0.494807
1       89   50   63  0.611684  0.640288   0.585526
2      173  120  293  0.455863  0.590444   0.371245
3      633  161  255  0.752675  0.797229   0.712838 




AttributeError: ignored

# Training

If you have access to the base data, you can train models with the script below.




In [ ]:
# constants (these will be abstracted away by inputs that you give to run)
PATH_TO_DATA_DIR = '' # specify this path


# -- model specific configurations
model_name = 'google/bigbird-roberta-base'
max_length = 1024

# -- training configurations
epochs = 5
batch_size = 2
verbose = 2
save_freq = 2

# -- dataset configurations
dataset_name = 'Persuade'

# -- experiment configurations
strategy = 'standard_bieo'
strat_name, strat_label = strategy.split('_')

# -- inferred configurations
df_label_map = LABELS_MAP_DICT[dataset_name][strat_label]
num_labels = len(set(df_label_map.label))
Processor = eval(f'{dataset_name}Processor')


### Tokenizer, Model and Optimizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name, add_prefix_space=True)
model = AutoModelForTokenClassification.from_pretrained(model_name, num_labels=num_labels) 
optimizer = torch.optim.Adam(params=model.parameters())

Some weights of the model checkpoint at google/bigbird-roberta-base were not used when initializing BigBirdForTokenClassification: ['cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BigBirdForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BigBirdForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BigBirdForTokenClassification were no

### Dataset 
Note this will change as the Processor develops. On the cluster you will need to use different options

In [ ]:
processor = Processor(PATH_TO_DATA_DIR)
processor = processor.from_json()
df_total = processor.dataframe

df_dict = processor.get_tts(test_size=0.3, val_size=0.1)
df_train = df_dict.get('train')[['text', 'labels']]
df_test = df_dict.get('test')[['text', 'labels']]
df_val = df_dict.get('val')[['text', 'labels']]


<class 'argminer.data.PersuadeProcessor'>


In [ ]:
train_set = ArgumentMiningDataset(df_label_map, df_train, tokenizer, max_length, strategy)
test_set = ArgumentMiningDataset(df_label_map, df_test, tokenizer, max_length, strategy, is_train=False)

train_loader = DataLoader(train_set, batch_size=batch_size)
test_loader = DataLoader(test_set, batch_size=batch_size)

In [ ]:
if not os.path.exists('models'):
  os.makedirs('models')
  print('models directory created!')
model.to(DEVICE)
print(f'Model pushed to device: {DEVICE}')
for epoch in range(epochs):
    model.train()
    start_epoch_message = f'EPOCH {epoch + 1} STARTED'
    print(start_epoch_message)
    print(f'{"-" * len(start_epoch_message)}')
    start_epoch = time.time()

    start_load = time.time()
    training_loss = 0
    for i, (inputs, targets) in enumerate(train_loader):
        start_train = time.time()
        inputs = move_to_device(inputs, DEVICE)
        targets = move_to_device(targets, DEVICE)
        if DEVICE != 'cpu':
            print(f'GPU Utilisation at batch {i+1} after data loading: {get_gpu_utilization()}')

        optimizer.zero_grad()

        loss, outputs = model(
            labels=targets,
            input_ids=inputs['input_ids'],
            attention_mask=inputs['attention_mask'],
            return_dict=False
        )
        if DEVICE != 'cpu':
            print(f'GPU Utilisation at batch {i+1} after training: {get_gpu_utilization()}')


        training_loss += loss.item()

        # backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        del targets, inputs, loss, outputs
        gc.collect()
        torch.cuda.empty_cache()

        end_train = time.time()

        if verbose > 1:
            print(
                f'Batch {i + 1} complete. Time taken: load({start_train - start_load:.3g}), '
                f'train({end_train - start_train:.3g}), total({end_train - start_load:.3g}). '
            )
        start_load = time.time()

    print_message = f'Epoch {epoch + 1}/{epochs} complete. ' \
                    f'Time taken: {start_load - start_epoch:.3g}. ' \
                    f'Loss: {training_loss/(i+1): .3g}'

    if verbose:
        print(f'{"-" * len(print_message)}')
        print(print_message)
        print(f'{"-" * len(print_message)}')

    if epoch % save_freq == 0:
        encoded_model_name = encode_model_name(model_name, epoch+1)
        save_path = f'models/{encoded_model_name}'
        model.save_pretrained(save_path)
        print(f'Model saved at epoch {epoch+1} at: {save_path}')

encoded_model_name = encode_model_name(model_name, 'final')
save_path = f'models/{encoded_model_name}'
model.save_pretrained(save_path)
print(f'Model saved at epoch {epoch + 1} at: {save_path}')

Model pushed to device: cuda
EPOCH 1 STARTED
---------------
GPU Utilisation at batch 1 after data loading: 2739


/usr/local/lib/python3.7/dist-packages/transformers/models/big_bird/modeling_big_bird.py:978: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  * num_indices_to_pick_from


GPU Utilisation at batch 1 after training: 5763
Batch 1 complete. Time taken: load(0.0248), train(2.19), total(2.22). 
GPU Utilisation at batch 2 after data loading: 2797
GPU Utilisation at batch 2 after training: 7251
Batch 2 complete. Time taken: load(0.0156), train(2.04), total(2.05). 
GPU Utilisation at batch 3 after data loading: 2797
GPU Utilisation at batch 3 after training: 7251
Batch 3 complete. Time taken: load(0.0133), train(2.03), total(2.04). 
GPU Utilisation at batch 4 after data loading: 2797
GPU Utilisation at batch 4 after training: 7251
Batch 4 complete. Time taken: load(0.0162), train(2.04), total(2.06). 
GPU Utilisation at batch 5 after data loading: 2797
GPU Utilisation at batch 5 after training: 7251
Batch 5 complete. Time taken: load(0.0172), train(2.03), total(2.04). 
GPU Utilisation at batch 6 after data loading: 2797
GPU Utilisation at batch 6 after training: 7251
Batch 6 complete. Time taken: load(0.024), train(2.02), total(2.05). 
GPU Utilisation at batch 7 

KeyboardInterrupt: ignored

In [ ]:
# load trained model
path = ''
trained_model = AutoModelForTokenClassification.from_pretrained(path)

In [ ]:
df_metrics, df_scores = inference(trained_model, test_loader)

In [ ]:
_get_scores_agg(df_scores)